# Definición de la neurona
Se define la estructura de la neurona, la cual contendrá toda la información necesaria para que pueda realizar las operaciones que requiere.

In [1]:
import random

class Neuron:
  def __init__(self, function, value=0.0, id=None):
    self.value = value
    self.id = id
    self.funct = function
    self.der_funct = lambda F_x: F_x * (1 - F_x)
    self.next_layer = []
    self.prev_layer = []
    self.curr_weights = []
    self.upd_weights = []
    self.bias = 0


  def add_layer(self, weights=None, bias=1, nodes_layer=None):
    self.prev_layer = nodes_layer
    
    for node in nodes_layer:
      node.next_layer.append(self)
      
    if weights != None:
      self.curr_weights = weights
    else:
      self.curr_weights = [random.random() for n in nodes_layer]

    self.bias = bias


  def calculate_value(self, apply_function=True):
    sum = 0.0
    for (index, node) in enumerate(self.prev_layer):
      sum += node.value * self.curr_weights[index]

    self.value = sum + self.bias

    if apply_function:
      self.value = self.funct(self.value)

    return self.value

## Probando la neurona
Se realizan algunas pruebas para verificar que todas las funciones las realiza de una forma correcta.
  Se crea una neurona que use la función sigmoide como función de activación, luego se le agrega una capa a partir de los valores que deberian tener los nodos. Por ultimo se calcula el valor activado de la neurona.

In [2]:
import math

def sigmoide(x):
  return 1 / (1 + math.exp(-x))

values = [0.34, 0.99, 0.03]
nodes = [Neuron(sigmoide, v) for v in values]

weights = [0.5, 4.7, 40.07]
biases = [0.1, 0.05, 0.08]

n = Neuron(sigmoide)
n.add_layer(weights, sum(biases), nodes)

n.calculate_value()

0.9980830478801297

# Definición de la red neuronal
Se defina la estructura de la red neuronal, la cual tendra las funciones básicas para que pueda operar.

In [20]:
class NeuronalNetwork:
  def __init__(self, cant_input, cant_output, function):
    self.input_layer = [Neuron(function, id=(0, i + 1)) for i in range(cant_input)]
    self.output_layer = [Neuron(function, id=(-1, i + 1)) for i in range(cant_output)]
    self.hidden_layers = []

  def is_empty(self):
    return len(self.hidden_layers) == 0

  def get_function(self):
    return self.input_layer[0].funct

  def add_layers(self, layers):
    # Se obtiene la malla
    self.hidden_layers = self._make_connections(layers)

    wi = [[0.1, 0.2, 0.15], [0.5, 0.01, 0.1]]
    wo = [[0.5, 0.6, 0.4], [0.3, 0.7, 0.1]]
    bi = [[0.2, 0.3, 0.01], [0.05, 0.03, 0.1]]
    bo = [[0.07, 0.1, 0.03], [0.009, 0.12, 0.023]]


    j = 0
    # Se conecta el principio de la malla con la capa de entrada
    for node in self.hidden_layers[0]:
      node.add_layer(nodes_layer=self.input_layer, weights=wi[j], bias=sum(bi[j]))
      j += 1

    j = 0
    # Se conecta la capa de salida con el final de la malla
    for node in self.output_layer:
      node.add_layer(nodes_layer=self.hidden_layers[-1], weights=wo[j], bias=sum(bo[j]))
      j += 1


  def fit(self, X, y):
    self.predict(X[0])
    deltas = self._create_delta_set()
    
    for (index, node) in enumerate(self.output_layer):
      value = node.funct(n.value)
      deltas[-1][index] = (value - y[index]) * node.der_funct(value)

    for (i_lay, layer) in enumerate(self.hidden_layers):
      for (i, node) in enumerate(layer):
        sum = 0.0

        for (j, next) in enumerate(node.next_layer):
          sum += deltas[i_lay + 2][j] * next.curr_weights[i]

        deltas[i_lay + 1][i] = sum * node.der_funct(node.value)

    print(deltas)

  
  def predict(self, input):
    # Se copian los valores del input en los nodos de la capa de entrada
    for (index, value) in enumerate(input):
      self.input_layer[index].value = value

    # Se calculan los valores de las capas ocultas
    for layer in self.hidden_layers:
      for node in layer:
        node.calculate_value()
        print(f'{node.id}: {node.value}')

    # Se calcula la salida sin aplicar la funcion de activacion al valor
    for node in self.output_layer:
      node.calculate_value(apply_function=False)

    return [node.value for node in self.output_layer]


  def _make_connections(self, layers):
    '''
    Realiza las conexiones entre las capas ingresadas, y retorna una malla de 
    la red neuronal (sin entrada ni salida)
    '''
    nodes_mesh = self._create_mesh(layers)
    length = len(layers)

    # For debug
    ws = [[0.9, 0.8], [1.2], [0.7], [2.4]]
    bs = [[0.01, 0.02], [0.04], [0.05], [0.03]]
    j = 0

    for i in range(1, length):
      for node in nodes_mesh[i]:
        node.add_layer(nodes_layer=nodes_mesh[i - 1], weights=ws[j], bias=sum(bs[j]))
        j += 1

    return nodes_mesh


  def _create_mesh(self, layers):
    '''
    Crea una malla de capas, pero sin conecciones entre ellas
    '''
    nodes_mesh = []
    function = self.get_function()

    for current_layer in layers:
      new_nodes_layer = [Neuron(function, id=(current_layer, j + 1)) for j in range(current_layer)] # for debug
      #new_nodes_layer = [current_layer for j in range(current_layer)]  # for debug
      #new_nodes_layer = [Neuron(function) for j in range(current_layer)]
      nodes_mesh.append(new_nodes_layer)

    return nodes_mesh


  def _create_delta_set(self):
    deltas_input = [0.0 for v in self.input_layer]
    deltas_hidden = [[0.0 for v in layer] for layer in self.hidden_layers]
    deltas_output = [0.0 for v in self.output_layer]

    return [deltas_input, *deltas_hidden, deltas_output]

In [21]:
layers = [2, 1, 3]

nn = NeuronalNetwork(3, 2, sigmoide)
nn.add_layers(layers)

In [22]:
X = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
y = [1, 2, 3]

nn.fit(X, y)

(2, 1): 0.8115326747861805
(2, 2): 0.7310585786300049
(1, 1): 0.7933480859247232
(3, 1): 0.7294862722276317
(3, 2): 0.646877897736161
(3, 3): 0.8736954059376705
[[0.0, 0.0, 0.0], [0.0, 0.0], [0.0], [-0.020016645249981164, -0.04720390815643218, -0.005095785575760461], [-0.05299811745092449, -0.2497841555938329]]


In [ ]:
# For debugging
def show_mesh(mesh):
  for layer in mesh:
    for node in layer:
      for (index, neight) in enumerate(node.prev_layer):
        print(f'{node.id} -> {neight.id}  w: {node.curr_weights[index]} v:{node.value}')

In [ ]:
# For debugging
def show_mesh_reverted(mesh):
  for layer in mesh:
    for (layer_index, node) in enumerate(layer):
      for (index, next_node) in enumerate(node.next_layer):
        print(f'{node.id} -> {next_node.id} w:{next_node.curr_weights[layer_index]}')

In [ ]:
d = 0.739304579579997 * 0.3 + 0.009 + 0.6535563064237191 * 0.7 + 0.12 + 0.8843939651086795 * 0.1 + 0.023
sigmoide(d)

0.7149850880113883

In [ ]:
x = [3, 4.7, 7.8]
print(nn.predict(x))

(2, 1): 0.9488262990829084
(2, 2): 0.9246310913535561
(1, 1): 0.8352976939290254
(3, 1): 0.739304579579997
(3, 2): 0.6535563064237191
(3, 3): 0.8843939651086795
[1.3155436596877017, 0.9197201848814704]


In [ ]:
mesh = [*nn.hidden_layers]
mesh.insert(0, nn.input_layer)
mesh.append(nn.output_layer)

In [ ]:
print('cantidad de capas totales:', len(mesh))

show_mesh(mesh)

cantidad de capas totales: 5
(2, 1) -> (0, 1)  w: 0.1 v:0.0
(2, 1) -> (0, 2)  w: 0.2 v:0.0
(2, 1) -> (0, 3)  w: 0.15 v:0.0
(2, 2) -> (0, 1)  w: 0.5 v:0.0
(2, 2) -> (0, 2)  w: 0.01 v:0.0
(2, 2) -> (0, 3)  w: 0.1 v:0.0
(1, 1) -> (2, 1)  w: 0.9 v:0.0
(1, 1) -> (2, 2)  w: 0.8 v:0.0
(3, 1) -> (1, 1)  w: 1.2 v:0.0
(3, 2) -> (1, 1)  w: 0.7 v:0.0
(3, 3) -> (1, 1)  w: 2.4 v:0.0
(-1, 1) -> (3, 1)  w: 0.5 v:0.0
(-1, 1) -> (3, 2)  w: 0.6 v:0.0
(-1, 1) -> (3, 3)  w: 0.4 v:0.0
(-1, 2) -> (3, 1)  w: 0.3 v:0.0
(-1, 2) -> (3, 2)  w: 0.7 v:0.0
(-1, 2) -> (3, 3)  w: 0.1 v:0.0


In [ ]:
show_mesh_reverted(mesh)

(0, 1) -> (2, 1) w:0.1
(0, 1) -> (2, 2) w:0.5
(0, 2) -> (2, 1) w:0.2
(0, 2) -> (2, 2) w:0.01
(0, 3) -> (2, 1) w:0.15
(0, 3) -> (2, 2) w:0.1
(2, 1) -> (1, 1) w:0.9
(2, 2) -> (1, 1) w:0.8
(1, 1) -> (3, 1) w:1.2
(1, 1) -> (3, 2) w:0.7
(1, 1) -> (3, 3) w:2.4
(3, 1) -> (-1, 1) w:0.5
(3, 1) -> (-1, 2) w:0.3
(3, 2) -> (-1, 1) w:0.6
(3, 2) -> (-1, 2) w:0.7
(3, 3) -> (-1, 1) w:0.4
(3, 3) -> (-1, 2) w:0.1
